# Estimativa preços de venda do dataset Boston_housing

Um dos problemas clássicos de machine learning é a previsão dos preços das casas de Boston apresentado no final da década de 1970.

Existe uma competição no Kaggle que utiliza este dataset:
https://www.kaggle.com/c/boston-housing

O objetivo desse notebook é fazer a previsão dos preços das casas utilizando uma rede neural com uma camada escondida de 40 neurônios.


## Importação dos pacotes

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
#from torch.optim.lr_scheduler import MultiStepLR, StepLR
from torch.autograd import Variable

import lib.pytorch_trainer as ptt

use_gpu = torch.cuda.is_available()
print('GPU available:', use_gpu)

GPU available: False


## Leitura do Dataset

Leitura do Dataset já normalizado: [boston_housing.ipynb](boston_housing.ipynb)
- **Atenção**: se houver erro em não achar o arquivo, executar o notebook boston_housing.ipynb

In [2]:
datain = np.load('../data/boston_housing_normalize.npz')

x, y = datain['Xtra'], datain['ytra']

In [3]:
n_samples, n_attributes = x.shape
print('n_samples:', n_samples)
print('n_attributes:', n_attributes)

n_samples: 506
n_attributes: 13


### Conversão para Tensor

In [4]:
x_train = torch.FloatTensor(x)
y_train = torch.FloatTensor(y)

## Rede, uma camada escondida de 40 neurônios

In [5]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(n_attributes, 40)
        self.ativ1  = nn.ReLU()
        self.layer2 = nn.Linear(40, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.ativ1(x)
        x = self.layer2(x)
        return x

model = Model()
if use_gpu:
    model.cuda()
model

Model (
  (layer1): Linear (13 -> 40)
  (ativ1): ReLU ()
  (layer2): Linear (40 -> 1)
)

In [6]:
class MyCallback(ptt.DeepNetTrainer):
    def __init__(self):
        self.metrics = {'mean':list([]), 'var':list([])}

    def on_train_begin(self, n_epochs, metrics):
        pass

    def on_train_end(self, n_epochs, metrics):
        plt.plot(np.arange(len(self.metrics['mean'])), self.metrics['mean'],
                 np.arange(len(self.metrics['var'])), self.metrics['var'])

    def on_epoch_begin(self, epoch, metrics):
        pass

    def on_epoch_end(self, epoch, metrics):
        self.metrics['mean'].append(self.trainer.model.state_dict()['layer1.weight'].mean())
        self.metrics['var'].append(self.trainer.model.state_dict()['layer1.weight'].var())

    def on_batch_begin(self, epoch, batch, mb_size):
        pass

    def on_batch_end(self, epoch, batch, y_pred, y_true, loss):
        pass

    def on_vbatch_begin(self, epoch, batch, mb_size):
        pass

    def on_vbatch_end(self, epoch, batch, y_pred, y_true, loss):
        pass

In [30]:
class EarlyStop(ptt.DeepNetTrainer):
    def __init__(self, patience):
        self.patience = patience
        self.patience_count = 0
        self.best_loss = 1e10

    def on_train_begin(self, n_epochs, metrics):
        pass

    def on_train_end(self, n_epochs, metrics):
        pass
    
    def on_epoch_begin(self, epoch, metrics):
        pass

    def on_epoch_end(self, epoch, metrics):
        if (metrics['valid']['losses'][-1] < self.best_loss):
            self.best_loss = metrics['valid']['losses'][-1]
            self.patience_count = 0
            print('Loss improved on validation!')
        else:
            self.patience_count += 1
            
        if (self.patience_count > self.patience):
            print('Early stopping!')
            raise KeyboardInterrupt
    
    def on_batch_begin(self, epoch, batch, mb_size):
        pass

    def on_batch_end(self, epoch, batch, y_pred, y_true, loss):
        pass

    def on_vbatch_begin(self, epoch, batch, mb_size):
        pass

    def on_vbatch_end(self, epoch, batch, y_pred, y_true, loss):
        pass

## Parâmetros do otimizador

A escolha dos parâmetros para o treinamento é crítica. A escolha do learning rate,
do método de ótimização, do tamanho do mini-batch, do número de camadas, do número de neurônios em
cada camada, são todas críticas para o sucesso do estimador.

In [31]:
criterion = nn.MSELoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.00001, momentum=0.9, nesterov=True)
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0005, alpha=0.7)
savebest = ptt.ModelCheckpoint('../../models/bostonhousing',reset=True, verbose=1)

callback = MyCallback()

earlystop = EarlyStop(2)

trainer = ptt.DeepNetTrainer(
        model =         model,
        criterion =     criterion,
        optimizer =     optimizer,
        #callbacks =     [savebest,ptt.PrintCallback()]
        callbacks =     [ptt.PrintCallback(), earlystop]
        )

## Treinamento

In [32]:
trainer.fit(50, x_train, y_train, valid_split=0.2, shuffle=True, batch_size=10)

Start training for 50 epochs
  1:   0.0s   T: 12.11414   V: 10.00125 best
Loss improved on validation!
  2:   0.0s   T: 12.07716   V: 9.80774 best
Loss improved on validation!
  3:   0.0s   T: 12.01750   V: 9.83229 
  4:   0.0s   T: 12.01036   V: 9.75776 best
Loss improved on validation!
  5:   0.0s   T: 11.94754   V: 9.85425 
  6:   0.0s   T: 11.95420   V: 9.79718 
  7:   0.0s   T: 11.91512   V: 9.73473 best
Loss improved on validation!
  8:   0.0s   T: 11.85589   V: 9.58090 best
Loss improved on validation!
  9:   0.0s   T: 11.84674   V: 9.65348 
 10:   0.0s   T: 11.81360   V: 9.86591 
 11:   0.0s   T: 11.78405   V: 9.69286 
Early stopping!
Stop training at epoch: 11/50


## Avaliação

In [ ]:
train_loss = trainer.metrics['train']['losses']
valid_loss = trainer.metrics['valid']['losses']
epochs = np.arange(len(train_loss))
plt.plot(epochs, train_loss,
         epochs, valid_loss)

In [ ]:
trainer.load_state('../../models/bostonhousing')

In [ ]:
eval = trainer.evaluate(x_train,y_train)
eval

In [ ]:
train_loss = trainer.metrics['train']['losses']
valid_loss = trainer.metrics['valid']['losses']
epochs = np.arange(len(train_loss))
plt.plot(epochs, train_loss,
         epochs, valid_loss)

In [ ]:
print('MSE:',valid_loss[-1])
print('RMSE:', np.sqrt(valid_loss[-1]))

# Exercícios

- Troque os hyperparâmetros para verificar se você consegue obter valores melhores. A forma de
  escolha e alteração de parâmetros é algo que se aprende com a experiência. Procure fazer
  sintonias finas, variando apenas um ou poucos parâmetros de cada vez.
  
  Parâmetros que podem ser trocados:
  - learning rate
  - n. de camadas
  - n. de neurônios
  - troca de função de ativação
  - método de otimizador do gradiente descendente
